In [1]:
#model套件
import torch
from torch import nn as nn
from torch.nn.parameter import Parameter as Parameter
from torch.utils.data import Dataset
from torch.autograd import Variable as Variable
from torchsummary import summary
import torch.nn.functional as F

#聲音處理套件
import librosa
#資料處理套件
import pandas as pd
import numpy as np
#時間套件
import time

from enum import Enum

In [2]:
#存檔位置
ModelMode = "LeNet5_ESC50_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
filePath_Train = ModelMode + TrainDataSoundMode[3] + "_LRup_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[3] +  "_LRup_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[3] + "_LRup"

In [3]:
#音訊資訊CSV檔位置
DATA_CSV_PATH = "D:/Hu_code/ESC-50-master/ESC-50-master/meta/esc50.csv"
#音訊存放位置
DATA_PATH = "D:/Hu_code/ESC-50-master/ESC-50-master/audio/"

def loadDataESC50(data_path, data_csv_path, soundLength):
    dataX_Fold = []
    dataY_Fold = []
    
    #讀取CSV檔
    df = pd.read_csv(data_csv_path)

    #計數
    count = 0
    
    for Fold in range(1, 6, 1):
        #初始化
        dataX = []
        dataY = []
        #篩選資料
        sound_df = df[df.fold == Fold]
        #轉換train資料/添加標籤
        for data in sound_df.itertuples():
            sig, sr = librosa.load(data_path+data[1])
            for i in range((6-soundLength)):
                sound = sig[i*sr : int((i+soundLength)*sr)]
                sound = audio_to_melspectrogram(audio=sound, sr=sr)
                dataX.append(sound)
                dataY.append(data[3])       
            count +=1
            print("{:.3f}".format(100 * count / len(df))+"%", end='\r')
        dataX_Fold.append(dataX)
        dataY_Fold.append(dataY)
                
    #通通轉成 torch 的 tensor 形式
    dataX_Fold = torch.Tensor(dataX_Fold)
    dataY_Fold = torch.Tensor(dataY_Fold).type(torch.LongTensor)
    
    #轉維度 FBCHW
    dataX_Fold = torch.reshape(dataX_Fold, (dataX_Fold.shape[0], dataX_Fold.shape[1], 1, dataX_Fold.shape[2], dataX_Fold.shape[3]))
           
    # 完成時打印最終消息
    print("Progress: 100% - Completed")
    return dataX_Fold, dataY_Fold

class conf:
    sr = 44100  #採樣率
    hop_length = 512  #帧移
    fmin = 20  #最低頻率
    fmax = sr // 2  #最高頻率
    n_mels = 128  #濾波器個數
    n_fft = 2048  #FFT窗口長度

#轉頻譜圖
def audio_to_melspectrogram(audio, sr):
    spectrogram = librosa.feature.melspectrogram(y=audio, 
                                                 sr=sr,
                                                 n_mels=128,
                                                 hop_length=512,
                                                 n_fft=2048,
                                                 fmin=20,
                                                 fmax=sr//2)
    spectrogram = librosa.power_to_db(spectrogram)
    return spectrogram

class CustomDatasetESC50(Dataset):
    def __init__(self, inputDataX, inputDataY, fold, isTrain):
        #防止範圍錯誤
        if(fold>5 or fold <1):
            raise ValueError("Fold範圍錯誤")
            
        #資料初始化
        self.dataX = []
        self.dataY = []
        
        # True:訓練資料  False:測試資料
        if isTrain:
            for Fold in range(1, 6, 1):
                if(Fold != fold):
                    for X in inputDataX[Fold-1]:
                        self.dataX.append(X)
                    for Y in inputDataY[Fold-1]:
                        self.dataY.append(Y)
        else:
            self.dataX = inputDataX[fold-1]
            self.dataY = inputDataY[fold-1]

    def __len__(self):
        return len(self.dataX)
    
    def __getitem__(self, index):
        # 根據索引返回單個樣本及其標籤
        data = self.dataX[index]
        label = self.dataY[index]
        
        return data, label

In [4]:
#LeNet5模型
class LeNet(nn.Module):
    def __init__(self, nums_class=10):
        super(LeNet,self).__init__()
        
        #卷基層
        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        )
        
        #全連接層
        self.fullconn_layer = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(32,120),
            nn.ReLU(),
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,nums_class)           
        )
        
    def forward(self,x):
        output = self.conv_layer(x)
        output = self.fullconn_layer(output)
        return output

In [5]:
#訓練模型(全部訓練)
def train_model_TrainAll(trainFold, setLR):
    #用來記錄全部訓練過程的準確度和loss
    train_loss_array = []
    train_accuracy_array = []
    
    #---------------------------------------------------
    #訓練相關參數
    #---------------------------------------------------
    #訓練次數
    num_epochs = 60
    #批次大小
    BATCH_SIZE = 64
    #學習率
    LR = setLR
    #loss方法
    loss_func = nn.CrossEntropyLoss()
    #優化器
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    #--------------------------------------------------------------------
    
    #--------------------------------------------------------------------
    #scheduler
    #--------------------------------------------------------------------
    #調度器
    scheduler  = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.5)
    #--------------------------------------------------------------------
    
    #--------------------------------------------------------------------
    #資料抓入
    #預處理
    #--------------------------------------------------------------------
    #資料抓入
    trainloader = CustomDatasetESC50(inputDataX = dataX,
                               inputDataY = dataY,
                               fold = trainFold,
                               isTrain = True)
    #分批方法
    train_loader = torch.utils.data.DataLoader(trainloader, batch_size = BATCH_SIZE, shuffle = True)
    #--------------------------------------------------------------------
    
    #訓練次數
    for epoch in range(num_epochs):
        #訓練準確度
        correct_train = 0
        #訓練資料總筆數
        total_train = 0
        #批次訓練
        for i, data in enumerate(train_loader):
            #開啟模型訓練
            model.train()
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            #清除梯度
            optimizer.zero_grad()
            #向前傳播
            outputs = model(inputs)
            #計算loss
            train_loss = loss_func(outputs, labels)
            #反向傳播
            train_loss.backward()
            #梯度下降
            optimizer.step()
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_train += len(labels)
            #計算判斷正確的筆數
            correct_train += (predicted == labels).float().sum()
        #計算這次訓練的準確度
        train_accuracy = 100 * correct_train / float(total_train)
        #紀錄這次訓練的準確度
        train_accuracy_array.append(train_accuracy)
        #紀錄這次訓練的loss
        train_loss_array.append(train_loss.data)
        
        #印出本次訓練結果
        print('Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}%'.format(epoch+1, num_epochs, train_loss.data, train_accuracy))
        #調度器走一步
        scheduler.step()
        
    #測試準確度
    test_accuracy1 = test_model(inputDataX = dataX1, inputDataY = dataY1, testFold = trainFold)
    test_accuracy3 = test_model(inputDataX = dataX3, inputDataY = dataY3, testFold = trainFold)
    test_accuracy5 = test_model(inputDataX = dataX5, inputDataY = dataY5, testFold = trainFold)
        
    #將gpu資料取回
    train_loss_array = torch.stack(train_loss_array,dim=-1).cpu()
    train_accuracy_array = torch.stack(train_accuracy_array,dim=-1).cpu()
    
    return train_loss_array, train_accuracy_array, test_accuracy1, test_accuracy3, test_accuracy5

In [6]:
#測試模型的準確度
def test_model(inputDataX, inputDataY, testFold):
    #固定模型不改變權重
    model.eval()
    #計數初始化
    correct = 0
    total = 0
    #資料抓取
    testloader = CustomDatasetESC50(inputDataX = inputDataX,
                               inputDataY = inputDataY,
                               fold = testFold,
                               isTrain = False)
    #封裝                                
    test_loader = torch.utils.data.DataLoader(testloader, batch_size=64, shuffle=False)
    
    #不計算梯度 加速計算
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            
            #預測結果
            outputs = model(inputs)
            
            #獲取預測標籤
            _, predicted = torch.max(outputs.data, 1)
            
            #計算準確度
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Test accuracy of the model: {:.6f}%'.format(100 * correct / total))
    return (100 * correct / total)

In [7]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
dataX1, dataY1 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 1)
print("讀取3秒資料")
dataX3, dataY3 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 3)
print("讀取5秒資料")
dataX5, dataY5 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料


C:\Users\test\AppData\Local\Temp\ipykernel_14904\2625783892.py:36: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  dataX_Fold = torch.Tensor(dataX_Fold)


Progress: 100% - Completed
讀取3秒資料
Progress: 100% - Completed
讀取5秒資料
Progress: 100% - Completed
Data Loading Time: 130.24 seconds


In [8]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []

#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    kTLTrainTime = []
    kTLTrainAccuracy = []
    kTLTestAccuracy1 = []
    kTLTestAccuracy3 = []
    kTLTestAccuracy5 = []

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = LeNet(nums_class = 50)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    
    #學習率
    lr=0
    for TL in range(3):
        print("----------------------------------------------------------------------------------------------------------------")
        print("fold: ", fold, "TL: ", TL)
        print("----------------------------------------------------------------------------------------------------------------")
        #訓練資料(要用幾秒訓練)
        if(TL==0):
            #1s
            dataX = dataX1
            dataY = dataY1
            lr = 0.0008
        elif(TL==1):
            #3s
            dataX = dataX3
            dataY = dataY3
            lr = 0.0016
        else:
            #5s
            dataX = dataX5
            dataY = dataY5
            lr = 0.0032

        #紀錄訓練開始時間
        start_time = time.time()

        #訓練模型(只訓練分類器)
        train_loss, train_accuracy, test_accuracy1, test_accuracy3, test_accuracy5 = train_model_TrainAll(trainFold=fold, setLR=lr)

        #紀錄訓練結束時間
        end_time = time.time()

        #計算訓練耗時
        training_time = end_time - start_time

        #印出訓練耗時
        print("Training Time: {:.2f} seconds".format(training_time))

        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("訓練過程\n")
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        for i in range(len(train_loss)):
            # 寫入資料
            f.write("Fold: {} TL: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, TL, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
        f.write("Training Time: {:.2f} seconds\n".format(training_time))
        f.write("---------------------------------------------------------------------------------------------------------------\n")

        #--------------------------------------------------------------------
        
        kTLTrainTime.append(training_time)
        kTLTrainAccuracy.append(train_accuracy[-1])
        kTLTestAccuracy1.append(test_accuracy1)
        kTLTestAccuracy3.append(test_accuracy3)
        kTLTestAccuracy5.append(test_accuracy5)
        
    kFoldTrainTime.append(kTLTrainTime)
    kFoldTrainAccuracy.append(kTLTrainAccuracy)
    kFoldTestAccuracy1.append(kTLTestAccuracy1)
    kFoldTestAccuracy3.append(kTLTestAccuracy3)
    kFoldTestAccuracy5.append(kTLTestAccuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

可用的 GPU 数量: 1
----------------------------------------------------------------------------------------------------------------
fold:  1 TL:  0
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/60 Train_Loss: 3.5126240253448486 Train_acc: 5.062500%
Train Epoch: 2/60 Train_Loss: 3.2122764587402344 Train_acc: 9.937500%
Train Epoch: 3/60 Train_Loss: 3.0619056224823 Train_acc: 17.162500%
Train Epoch: 4/60 Train_Loss: 2.6372828483581543 Train_acc: 23.000002%
Train Epoch: 5/60 Train_Loss: 2.9422049522399902 Train_acc: 26.150002%
Train Epoch: 6/60 Train_Loss: 2.3903615474700928 Train_acc: 28.962502%
Train Epoch: 7/60 Train_Loss: 2.28421688079834 Train_acc: 32.125000%
Train Epoch: 8/60 Train_Loss: 2.5698869228363037 Train_acc: 34.587502%
Train Epoch: 9/60 Train_Loss: 2.0844805240631104 Train_acc: 34.837502%
Train Epoch: 10/60 Train_Loss: 2.0431387424468994 Train_acc: 37.775002%
Train Epoch: 11/60 Train_Loss: 2.0029773

Train Epoch: 47/60 Train_Loss: 0.45230725407600403 Train_acc: 86.958336%
Train Epoch: 48/60 Train_Loss: 0.38907596468925476 Train_acc: 86.458336%
Train Epoch: 49/60 Train_Loss: 0.518805980682373 Train_acc: 87.083336%
Train Epoch: 50/60 Train_Loss: 0.4938589930534363 Train_acc: 86.666672%
Train Epoch: 51/60 Train_Loss: 0.6385607719421387 Train_acc: 86.812500%
Train Epoch: 52/60 Train_Loss: 0.6880526542663574 Train_acc: 87.291672%
Train Epoch: 53/60 Train_Loss: 0.4669197201728821 Train_acc: 87.541672%
Train Epoch: 54/60 Train_Loss: 0.6084859371185303 Train_acc: 87.250000%
Train Epoch: 55/60 Train_Loss: 0.507175862789154 Train_acc: 87.666672%
Train Epoch: 56/60 Train_Loss: 0.5673734545707703 Train_acc: 87.229172%
Train Epoch: 57/60 Train_Loss: 0.3886697292327881 Train_acc: 87.729172%
Train Epoch: 58/60 Train_Loss: 0.34544917941093445 Train_acc: 87.291672%
Train Epoch: 59/60 Train_Loss: 0.47673341631889343 Train_acc: 87.250000%
Train Epoch: 60/60 Train_Loss: 0.44635117053985596 Train_acc: 

Train Epoch: 31/60 Train_Loss: 1.4683619737625122 Train_acc: 54.087502%
Train Epoch: 32/60 Train_Loss: 1.4682570695877075 Train_acc: 54.787502%
Train Epoch: 33/60 Train_Loss: 1.8331037759780884 Train_acc: 55.000004%
Train Epoch: 34/60 Train_Loss: 1.5003466606140137 Train_acc: 54.700001%
Train Epoch: 35/60 Train_Loss: 1.4643522500991821 Train_acc: 54.762501%
Train Epoch: 36/60 Train_Loss: 1.578352689743042 Train_acc: 55.450001%
Train Epoch: 37/60 Train_Loss: 1.4812095165252686 Train_acc: 54.837502%
Train Epoch: 38/60 Train_Loss: 1.631751537322998 Train_acc: 55.725002%
Train Epoch: 39/60 Train_Loss: 1.5363025665283203 Train_acc: 55.750004%
Train Epoch: 40/60 Train_Loss: 1.5802597999572754 Train_acc: 55.375004%
Train Epoch: 41/60 Train_Loss: 1.5668803453445435 Train_acc: 56.400002%
Train Epoch: 42/60 Train_Loss: 1.5679211616516113 Train_acc: 56.650002%
Train Epoch: 43/60 Train_Loss: 1.582066297531128 Train_acc: 56.875004%
Train Epoch: 44/60 Train_Loss: 1.5617280006408691 Train_acc: 57.400

Train Epoch: 15/60 Train_Loss: 0.515968918800354 Train_acc: 78.937500%
Train Epoch: 16/60 Train_Loss: 0.3950358033180237 Train_acc: 85.000000%
Train Epoch: 17/60 Train_Loss: 0.5478287935256958 Train_acc: 88.000000%
Train Epoch: 18/60 Train_Loss: 0.32287272810935974 Train_acc: 87.625000%
Train Epoch: 19/60 Train_Loss: 0.2942063510417938 Train_acc: 89.750000%
Train Epoch: 20/60 Train_Loss: 0.32650792598724365 Train_acc: 91.312500%
Train Epoch: 21/60 Train_Loss: 0.28558433055877686 Train_acc: 90.312500%
Train Epoch: 22/60 Train_Loss: 0.6074747443199158 Train_acc: 90.437500%
Train Epoch: 23/60 Train_Loss: 0.25398021936416626 Train_acc: 90.937500%
Train Epoch: 24/60 Train_Loss: 0.5356370210647583 Train_acc: 88.500000%
Train Epoch: 25/60 Train_Loss: 0.25900861620903015 Train_acc: 86.437500%
Train Epoch: 26/60 Train_Loss: 0.24797925353050232 Train_acc: 88.875000%
Train Epoch: 27/60 Train_Loss: 0.6210179328918457 Train_acc: 89.687500%
Train Epoch: 28/60 Train_Loss: 0.6183210015296936 Train_acc

Train Epoch: 1/60 Train_Loss: 1.4741554260253906 Train_acc: 45.604168%
Train Epoch: 2/60 Train_Loss: 1.4698225259780884 Train_acc: 55.604168%
Train Epoch: 3/60 Train_Loss: 1.2279256582260132 Train_acc: 57.770836%
Train Epoch: 4/60 Train_Loss: 1.6278141736984253 Train_acc: 58.895836%
Train Epoch: 5/60 Train_Loss: 1.386833906173706 Train_acc: 57.937500%
Train Epoch: 6/60 Train_Loss: 0.9787562489509583 Train_acc: 59.166668%
Train Epoch: 7/60 Train_Loss: 1.1607391834259033 Train_acc: 62.729168%
Train Epoch: 8/60 Train_Loss: 1.200997233390808 Train_acc: 62.041668%
Train Epoch: 9/60 Train_Loss: 0.8480714559555054 Train_acc: 61.416668%
Train Epoch: 10/60 Train_Loss: 1.1271541118621826 Train_acc: 64.354172%
Train Epoch: 11/60 Train_Loss: 1.0783631801605225 Train_acc: 62.395836%
Train Epoch: 12/60 Train_Loss: 1.1096436977386475 Train_acc: 63.416668%
Train Epoch: 13/60 Train_Loss: 1.0655394792556763 Train_acc: 64.354172%
Train Epoch: 14/60 Train_Loss: 1.3605365753173828 Train_acc: 68.541672%
Tra

Train Epoch: 50/60 Train_Loss: 0.0911354199051857 Train_acc: 98.000000%
Train Epoch: 51/60 Train_Loss: 0.061288394033908844 Train_acc: 97.937500%
Train Epoch: 52/60 Train_Loss: 0.1529352068901062 Train_acc: 97.562500%
Train Epoch: 53/60 Train_Loss: 0.06381373107433319 Train_acc: 98.437500%
Train Epoch: 54/60 Train_Loss: 0.0780891478061676 Train_acc: 98.062500%
Train Epoch: 55/60 Train_Loss: 0.1376015692949295 Train_acc: 98.062500%
Train Epoch: 56/60 Train_Loss: 0.12964628636837006 Train_acc: 97.375000%
Train Epoch: 57/60 Train_Loss: 0.10126993805170059 Train_acc: 98.312500%
Train Epoch: 58/60 Train_Loss: 0.11050250381231308 Train_acc: 98.250000%
Train Epoch: 59/60 Train_Loss: 0.09656744450330734 Train_acc: 97.500000%
Train Epoch: 60/60 Train_Loss: 0.14466539025306702 Train_acc: 98.062500%
Test accuracy of the model: 30.800000%
Test accuracy of the model: 44.083333%
Test accuracy of the model: 53.250000%
Training Time: 34.55 seconds
可用的 GPU 数量: 1
----------------------------------------

Train Epoch: 34/60 Train_Loss: 0.4642806947231293 Train_acc: 83.104172%
Train Epoch: 35/60 Train_Loss: 0.4596680998802185 Train_acc: 83.395836%
Train Epoch: 36/60 Train_Loss: 0.8647333383560181 Train_acc: 83.395836%
Train Epoch: 37/60 Train_Loss: 0.6588723063468933 Train_acc: 83.812500%
Train Epoch: 38/60 Train_Loss: 0.466327041387558 Train_acc: 82.541672%
Train Epoch: 39/60 Train_Loss: 0.5802727937698364 Train_acc: 83.145836%
Train Epoch: 40/60 Train_Loss: 0.5328310132026672 Train_acc: 82.687500%
Train Epoch: 41/60 Train_Loss: 0.5984652042388916 Train_acc: 84.104172%
Train Epoch: 42/60 Train_Loss: 0.46104294061660767 Train_acc: 83.958336%
Train Epoch: 43/60 Train_Loss: 0.6141953468322754 Train_acc: 83.229172%
Train Epoch: 44/60 Train_Loss: 0.3792143166065216 Train_acc: 83.729172%
Train Epoch: 45/60 Train_Loss: 0.37038901448249817 Train_acc: 84.625000%
Train Epoch: 46/60 Train_Loss: 0.45310619473457336 Train_acc: 86.750000%
Train Epoch: 47/60 Train_Loss: 0.5384572744369507 Train_acc: 8

Train Epoch: 17/60 Train_Loss: 2.0287184715270996 Train_acc: 46.725002%
Train Epoch: 18/60 Train_Loss: 1.9888354539871216 Train_acc: 47.212502%
Train Epoch: 19/60 Train_Loss: 1.8826276063919067 Train_acc: 47.887501%
Train Epoch: 20/60 Train_Loss: 1.8424445390701294 Train_acc: 48.950001%
Train Epoch: 21/60 Train_Loss: 1.7077692747116089 Train_acc: 48.637501%
Train Epoch: 22/60 Train_Loss: 1.7713463306427002 Train_acc: 49.900002%
Train Epoch: 23/60 Train_Loss: 1.9847298860549927 Train_acc: 49.950001%
Train Epoch: 24/60 Train_Loss: 1.7494933605194092 Train_acc: 50.437504%
Train Epoch: 25/60 Train_Loss: 1.394260287284851 Train_acc: 51.037502%
Train Epoch: 26/60 Train_Loss: 1.3584383726119995 Train_acc: 51.750004%
Train Epoch: 27/60 Train_Loss: 1.8199172019958496 Train_acc: 52.162502%
Train Epoch: 28/60 Train_Loss: 1.8759191036224365 Train_acc: 51.875004%
Train Epoch: 29/60 Train_Loss: 1.6861176490783691 Train_acc: 52.662502%
Train Epoch: 30/60 Train_Loss: 1.3316603899002075 Train_acc: 53.7

Train Epoch: 1/60 Train_Loss: 2.802293062210083 Train_acc: 25.062500%
Train Epoch: 2/60 Train_Loss: 1.9695346355438232 Train_acc: 36.687500%
Train Epoch: 3/60 Train_Loss: 1.4056320190429688 Train_acc: 51.750000%
Train Epoch: 4/60 Train_Loss: 1.1524454355239868 Train_acc: 65.875000%
Train Epoch: 5/60 Train_Loss: 0.9534382224082947 Train_acc: 68.375000%
Train Epoch: 6/60 Train_Loss: 0.8743143081665039 Train_acc: 73.062500%
Train Epoch: 7/60 Train_Loss: 1.342803716659546 Train_acc: 68.750000%
Train Epoch: 8/60 Train_Loss: 0.6449741721153259 Train_acc: 68.500000%
Train Epoch: 9/60 Train_Loss: 0.6010019183158875 Train_acc: 72.062500%
Train Epoch: 10/60 Train_Loss: 1.1382217407226562 Train_acc: 67.437500%
Train Epoch: 11/60 Train_Loss: 0.6816624402999878 Train_acc: 71.687500%
Train Epoch: 12/60 Train_Loss: 1.263930082321167 Train_acc: 75.812500%
Train Epoch: 13/60 Train_Loss: 1.0696827173233032 Train_acc: 72.687500%
Train Epoch: 14/60 Train_Loss: 0.9899860620498657 Train_acc: 72.937500%
Trai

In [9]:
class FilterMode(Enum):
    Max = "Max"
    Min = "Min"
    Mean = "Mean"
    
def listFilter(data, mode:FilterMode):
    TL = []
    for fold in range(len(data)):
        TL.append(data[fold][len(data[fold])-1])
    
    switch = {
        FilterMode.Max: max(TL),
        FilterMode.Min: min(TL),
        FilterMode.Mean: np.mean(TL)
    }

    return switch.get(mode, -1)

#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    for TL in range(3):
        # 寫入資料
        f.write("Fold: {} TL: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, TL, kFoldTrainAccuracy[fold][TL], kFoldTestAccuracy1[fold][TL], kFoldTestAccuracy3[fold][TL], kFoldTestAccuracy5[fold][TL], kFoldTrainTime[fold][TL]))
    f.write("-----------------------------------------------------------------------------------\n")
    
# 寫入資料[]
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Max), listFilter(kFoldTestAccuracy1, FilterMode.Max), listFilter(kFoldTestAccuracy3, FilterMode.Max), listFilter(kFoldTestAccuracy5, FilterMode.Max)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Min), listFilter(kFoldTestAccuracy1, FilterMode.Min), listFilter(kFoldTestAccuracy3, FilterMode.Min), listFilter(kFoldTestAccuracy5, FilterMode.Min)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Mean), listFilter(kFoldTestAccuracy1, FilterMode.Mean), listFilter(kFoldTestAccuracy3, FilterMode.Mean), listFilter(kFoldTestAccuracy5, FilterMode.Mean)))

f.close()